In [1]:
# Third party imports
from IPython.display import display, HTML
# Local imports
from libcomcat.dataframes import (get_detail_data_frame, get_dyfi_data_frame,
                                  get_history_data_frame, get_magnitude_data_frame,
                                  get_pager_data_frame, get_phase_dataframe,
                                  get_summary_data_frame)
from libcomcat.search import search, get_event_by_id


In [2]:
import pandas as pd

(Returns NoneType if no pager product)

In [3]:
df = pd.read_csv('../DASHBOARD/CSV_TRANSFORMADOS/Datos_USA.csv')

In [11]:
df.head()

,id,Magnitud,Primer reg,Ultimo registro,properties.felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,ids,ns,Dist Horizontal epicentro,properties.rms,Brecha azimutal,estado,Longitud,Latitud,Profundidad
0,us6000kbl9,2.50,2023-05-13 00:22:34.350,2023-05-13 01:02:32.040,NaN,NaN,NaN,0,96,",us6000kbl9,",10.0,0.46000,0.5100,119.00,Kansas,-99.499500,39.235400,3.69
1,nn00859459,2.80,2023-05-12 22:29:14.464,2023-05-12 23:43:48.040,NaN,NaN,NaN,0,121,",us6000kbkj,nn00859459,",14.0,0.22000,0.1563,64.37,Nevada,-118.914500,38.402200,11.50
2,nc73887311,2.91,2023-05-12 20:57:31.130,2023-05-12 23:55:14.063,NaN,NaN,NaN,0,130,",nc73887311,us6000kbjl,",24.0,0.10920,0.1100,182.00,California,-121.261330,35.747665,3.66
3,nc73887306,2.90,2023-05-12 20:57:24.320,2023-05-12 23:54:10.059,NaN,NaN,NaN,0,129,",nc73887306,",12.0,0.10120,0.1200,180.00,California,-121.248833,35.747501,4.59
4,ci40223751,2.77,2023-05-12 17:30:47.620,2023-05-12 23:47:37.500,NaN,NaN,NaN,0,118,",ci40223751,us6000kbhy,",94.0,0.07689,0.1400,20.00,California,-116.735167,33.979167,17.35


In [10]:
extra_data = []
for id in df['id']:
    pager_event = get_event_by_id(id)
    pager_country_df = get_pager_data_frame(pager_event, get_losses=True)
    if pager_country_df is not None:
        display(HTML(pager_country_df.to_html()))
    #     extra_data.append(pager_country_df)
# extra_data_df = pd.concat(extra_data)
# df = pd.merge(df, extra_data_df, on='id', how='left')

,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,mmi2,mmi3,mmi4,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,nc73887046,"4km NW of Canyondam, CA",2023-05-12 10:18:41.310,40.196,-121.099833,6.06,5.16,Total,1,368132,697998,3455312,25057,6202,1355,0,0,0,0,0,1,1485,4872


,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,mmi2,mmi3,mmi4,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,nc73886731,Lake Almanor Earthquake,2023-05-11 23:19:41.990,40.204167,-121.1095,5.85,5.48,Total,1,1247,1269204,3143680,100735,8396,1293,0,0,0,0,0,1,1417,4649


,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,mmi2,mmi3,mmi4,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,ci40215735,"7km WSW of Niland, CA",2023-04-30 07:58:19.380,33.203667,-115.585833,1.89,4.26,Total,1,10789895,1688521,102579,12110,24,0,0,0,0,0,0,2,0,2


,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,mmi2,mmi3,mmi4,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,ci40215583,"9km SW of Niland, CA",2023-04-30 07:10:10.180,33.189333,-115.593333,1.88,4.29,Total,1,10759525,1089418,562339,39281,59,0,0,0,0,0,0,2,0,2


,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,mmi2,mmi3,mmi4,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,ci40215575,"8km WSW of Niland, CA",2023-04-30 07:09:34.500,33.200833,-115.59,2.04,4.33,Total,1,4384600,6897174,1419211,40539,117,0,0,0,0,0,0,2,0,2


...

,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,mmi2,mmi3,mmi4,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,ci40151807,"16km N of Borrego Springs, CA",2022-12-31 12:12:26.650,33.3975,-116.393333,3.88,4.14,Total,1,15609653,8621053,950468,1743,0,0,0,0,0,0,0,2,0,2


,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,mmi2,mmi3,mmi4,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,nc73824236,"8km E of Hydesville, CA",2022-12-24 19:33:44.190,40.557167,-124.008333,22.4,4.16,Total,1,1020357,29972,105554,19044,0,0,0,0,0,0,0,1,0,2


,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,mmi2,mmi3,mmi4,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,nc73822026,"6km WSW of Ferndale, CA",2022-12-20 22:06:34.380,40.5455,-124.329167,19.74,4.35,Total,1,455502,180995,99524,30131,0,0,0,0,0,0,0,1,0,2


,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,mmi2,mmi3,mmi4,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,nc73821636,"6km NNE of Hydesville, CA",2022-12-20 15:09:05.740,40.6005,-124.068833,22.5,3.99,Total,1,1036669,30279,96851,22659,0,0,0,0,0,0,0,1,0,2


KeyboardInterrupt: 

In [13]:
dic = { "peru": {"minlatitude": -18.521, "maxlatitude": 0.132, "minlongitude": -84.419, "maxlongitude": -68.599},
            "usa": {"minlatitude": 24.6, "maxlatitude": 50, "minlongitude": -125, "maxlongitude": -65},
            "japon": {"minlatitude": 24.40, "maxlatitude": 45, "minlongitude": 122.93, "maxlongitude": 153.99}
    }
minlatitude, maxlatiude, minlongitude, maxlongitude = dic["usa"].keys
minlatitude

TypeError: cannot unpack non-iterable builtin_function_or_method object

In [ ]:
df.head()

In [14]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [16]:

def usgs_json_raw(url, filename, pais, format=None, alertlevel=None):
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=50*365)).strftime('%Y-%m-%dT%H:%M:%S')

    dic = {
        "peru": {"minlatitude": -18.521, "maxlatitude": 0.132, "minlongitude": -84.419, "maxlongitude": -68.599},
        "usa": {"minlatitude": 24.6, "maxlatitude": 50, "minlongitude": -125, "maxlongitude": -65},
        "japon": {"minlatitude": 24.40, "maxlatitude": 45, "minlongitude": 122.93, "maxlongitude": 153.99}
    }
    values = dic[pais]
    minlatitude = values["minlatitude"]
    maxlatitude = values["maxlatitude"]
    minlongitude = values["minlongitude"]
    maxlongitude = values["maxlongitude"]
    
    params = {
        "format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 2.5,
        "minlatitude": minlatitude, "maxlatitude": maxlatitude, "minlongitude": minlongitude,
        "maxlongitude": maxlongitude, "limit": 20000
    }
    
    if alertlevel:
        alert_levels = ["green", "yellow", "orange", "red"]
        dfs = []
        
        for level in alert_levels:
            params["alertlevel"] = level
            response = requests.get(url, params=params)
            data = response.json()
            features = data['features']
            df_sismo = pd.json_normalize(features)
            df_sismo["properties.time"] = pd.to_datetime(df_sismo["properties.time"], unit="ms")
            df_sismo["properties.updated"] = pd.to_datetime(df_sismo["properties.updated"], unit="ms")
            df_sismo[["Longitud", "Latitud", "Profundidad"]] = pd.DataFrame(df_sismo["geometry.coordinates"].tolist())
            dfs.append(df_sismo)
        
        merged_df = pd.concat(dfs)
        merged_df.to_csv(path_or_buf='../DASHBOARD/CSV_ORIGINAL/' + filename, index=False)
        return merged_df
    
    else:
        response = requests.get(url, params=params)
        data = response.json()
        features = data['features']
        df_sismo = pd.json_normalize(features)
        df_sismo["properties.time"] = pd.to_datetime(df_sismo["properties.time"], unit="ms")
        df_sismo["properties.updated"] = pd.to_datetime(df_sismo["properties.updated"], unit="ms")
        df_sismo[["Longitud", "Latitud", "Profundidad"]] = pd.DataFrame(df_sismo["geometry.coordinates"].tolist())
        df_sismo.to_csv(path_or_buf='../DASHBOARD/CSV_ORIGINAL/' + filename, index=False)
        return df_sismo

url = r'https://earthquake.usgs.gov/fdsnws/event/1/query?'
# filename = 'raw_usgs_pager_usa.csv'

usgs_json_raw(url, 'raw_alert_usa.csv', "usa", alertlevel=True)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates,Longitud,Latitud,Profundidad
0,Feature,nc73887046,5.16,"4km NW of Canyondam, CA",2023-05-12 10:18:41.310,2023-05-15 14:53:47.275,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,1347.0,...,0.19,37.0,mw,earthquake,"M 5.2 - 4km NW of Canyondam, CA",Point,"[-121.0998333, 40.196, 6.06]",-121.099833,40.196000,6.060
1,Feature,nc73886731,5.48,Lake Almanor Earthquake,2023-05-11 23:19:41.990,2023-05-16 13:39:33.501,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,7013.0,...,0.16,46.0,mw,earthquake,M 5.5 - Lake Almanor Earthquake,Point,"[-121.1095, 40.2041667, 5.85]",-121.109500,40.204167,5.850
2,Feature,ci40215735,4.26,"7km WSW of Niland, CA",2023-04-30 07:58:19.380,2023-05-12 09:29:19.854,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,19.0,...,0.26,90.0,mw,earthquake,"M 4.3 - 7km WSW of Niland, CA",Point,"[-115.5858333, 33.2036667, 1.89]",-115.585833,33.203667,1.890
3,Feature,ci40215583,4.29,"9km SW of Niland, CA",2023-04-30 07:10:10.180,2023-05-14 05:21:36.079,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,5.0,...,0.29,87.0,mw,earthquake,"M 4.3 - 9km SW of Niland, CA",Point,"[-115.5933333, 33.1893333, 1.88]",-115.593333,33.189333,1.880
4,Feature,ci40215575,4.33,"8km WSW of Niland, CA",2023-04-30 07:09:34.500,2023-05-12 03:32:21.944,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,50.0,...,0.25,88.0,mw,earthquake,"M 4.3 - 8km WSW of Niland, CA",Point,"[-115.59, 33.2008333, 2.04]",-115.590000,33.200833,2.040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,Feature,ci12456384,5.70,"13km WSW of Delta, B.C., MX",2002-02-22 19:32:41.750,2022-05-02 15:45:48.377,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,3281.0,...,0.38,217.5,mw,earthquake,"M 5.7 - 13km WSW of Delta, B.C., MX",Point,"[-115.3215, 32.3188333, 7.007]",-115.321500,32.318833,7.007
15,Feature,ci3310205,5.80,"3km NW of Brawley, CA",1979-10-16 06:58:43.450,2022-04-27 14:51:22.762,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,0.17,79.0,ml,earthquake,"M 5.8 - 3km NW of Brawley, CA",Point,"[-115.5575, 32.9986667, 14.19]",-115.557500,32.998667,14.190
0,Feature,nc73821036,6.40,"15km WSW of Ferndale, CA",2022-12-20 10:34:24.770,2023-05-16 10:44:34.492,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,5945.0,...,0.18,214.0,mw,earthquake,"M 6.4 - 15km WSW of Ferndale, CA",Point,"[-124.423, 40.525, 17.91]",-124.423000,40.525000,17.910
0,Feature,nc72282711,6.02,South Napa,2014-08-24 10:20:44.070,2022-05-03 18:17:55.653,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,16679.0,...,0.18,28.0,mw,earthquake,M 6.0 - South Napa,Point,"[-122.3123333, 38.2151667, 11.12]",-122.312333,38.215167,11.120


In [20]:
usgs_json_raw(url, "raw_alert_japon.csv", "japon", alertlevel=True)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates,Longitud,Latitud,Profundidad
0,Feature,us6000kbtz,5.6,"144 km SSE of Shimoda, Japan",2023-05-14 10:11:35.380,2023-05-15 10:18:19.469,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,1.0,...,0.70,33.0,mww,earthquake,"M 5.6 - 144 km SSE of Shimoda, Japan",Point,"[139.3129, 33.4097, 13.71]",139.3129,33.4097,13.710
1,Feature,us6000kbtf,5.8,"152 km SSE of Shimoda, Japan",2023-05-14 08:21:41.106,2023-05-15 15:17:57.040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,1.00,31.0,mww,earthquake,"M 5.8 - 152 km SSE of Shimoda, Japan",Point,"[139.3432, 33.3438, 5.938]",139.3432,33.3438,5.938
2,Feature,us6000kbta,5.5,"148 km SSE of Shimoda, Japan",2023-05-14 08:11:57.241,2023-05-15 08:18:12.295,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.0,...,0.59,42.0,mww,earthquake,"M 5.5 - 148 km SSE of Shimoda, Japan",Point,"[139.3982, 33.3953, 14.275]",139.3982,33.3953,14.275
3,Feature,us6000k9ue,5.9,"Hokkaido, Japan region",2023-05-05 17:47:16.549,2023-05-06 17:53:42.064,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,5.0,...,0.66,30.0,mww,earthquake,"M 5.9 - Hokkaido, Japan region",Point,"[141.9956, 41.5046, 62.297]",141.9956,41.5046,62.297
4,Feature,us6000k9rh,5.6,"43 km NE of Anamizu, Japan",2023-05-05 12:58:03.883,2023-05-15 05:47:45.143,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,8.0,...,0.65,50.0,mwb,earthquake,"M 5.6 - 43 km NE of Anamizu, Japan",Point,"[137.2737, 37.4882, 10]",137.2737,37.4882,10.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,Feature,us20007fta,6.2,"6 km S of Kurayoshi, Japan",2016-10-21 05:07:22.990,2023-02-08 23:44:57.663,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,171.0,...,0.75,23.0,mww,earthquake,"M 6.2 - 6 km S of Kurayoshi, Japan",Point,"[133.8092, 35.3743, 5.63]",133.8092,35.3743,5.630
0,Feature,us6000h519,7.3,"57 km ENE of Namie, Japan",2022-03-16 14:36:30.997,2023-03-10 20:56:20.662,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,467.0,...,0.88,29.0,mww,earthquake,"M 7.3 - 57 km ENE of Namie, Japan",Point,"[141.5793, 37.7132, 41]",141.5793,37.7132,41.000
1,Feature,us1000eu1c,5.5,"near the south coast of western Honshu, Japan",2018-06-17 22:58:34.770,2022-05-03 19:19:25.156,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,531.0,...,0.70,22.0,mww,earthquake,M 5.5 - near the south coast of western Honshu...,Point,"[135.6389, 34.8246, 10.34]",135.6389,34.8246,10.340
2,Feature,us20005iis,7.0,"6 km ESE of Kumamoto, Japan",2016-04-15 16:25:06.220,2022-12-23 11:51:30.843,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,81.0,...,0.85,32.0,mww,earthquake,"M 7.0 - 6 km ESE of Kumamoto, Japan",Point,"[130.7543, 32.7906, 10]",130.7543,32.7906,10.000
